In [ ]:
from pathlib import Path
import os.path

In [ ]:
out_dir = Path("./figure_mitosis_trajs_viz_Nikon_and_CXA_3-4_2024")
out_dir.mkdir(exist_ok=True)

In [ ]:
from livecellx.track.classify_utils import load_class2samples_from_json_dir, load_all_json_dirs


sample_json_dirs_v0 = [Path(r"../datasets/test_scs_EBSS_starvation/XY1/annotations"), Path(r"./datasets/test_scs_EBSS_starvation/XY16/annotations")]
round1_json_dirs = sample_json_dirs_v0 + [
    Path(r"../datasets/mitosis-annotations-2023/shiman_XY01/XY01"),
Path(r"../datasets/mitosis-annotations-2023/shiman_XY09/XY09"),
Path(r"../datasets/mitosis-annotations-2023/shiman_XY10/XY10"),
Path(r"../datasets/mitosis-annotations-2023/Yajushi/tifs_CFP_A549-VIM_lessThan24hr_NoTreat_NA_YL_Ti2e_2022-10-19/XY1/annotations"),
]

round2_json_dirs = [
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C0.5^4/"),
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C0.75^4/"),
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^3/"),\
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^4/")
] + [
    Path(f"../datasets/mitosis-annotations-2023/Gaohan_tifs_CFP_A549-VIM_lessThan24hr_NoTreat_NA_YL_Ti2e_2022-10-19/XY{pos}/annotations") for pos in range(4, 14)
]

cxa_annotation_dirs = [
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C0.5^4/"),
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C0.75^4/"),
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^3/"),\
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^4/")
] + [
    Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_normal/0.25^4/"),
    Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_normal/C0.5^4/"),
    Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_normal/C10^3/")
]
sample_json_dirs = set(sample_json_dirs_v0 + round1_json_dirs + round2_json_dirs + cxa_annotation_dirs)
all_class2samples, all_class2sample_extra_info = load_all_json_dirs(sample_json_dirs)

In [ ]:
all_class2samples["mitosis"][0][0].meta

In [ ]:
for key in all_class2samples:
    print(key, len(all_class2samples[key]))

In [ ]:
mitosis_sampels = all_class2samples["mitosis"]
mitosis_sampels[0][0].meta

check if sc has image data for future feature calculation

In [ ]:
# for idx, sample in enumerate(mitosis_sampels):
#     for sc in sample:
#         sc.get_img()

Store sample index in all_class2samples to each single cell for future use

In [ ]:

import numpy as np


for idx, sample in enumerate(mitosis_sampels):
    if len(sample) == 0:
        continue
    _first_time = sample[0].timeframe
    _time2scs = {}
    for sc in sample:
        if sc.timeframe not in _time2scs:
            _time2scs[sc.timeframe] = []
        _time2scs[sc.timeframe].append(sc)
    
    # find the smallest time with more than one scs
    _first_div_time = None
    for time in sorted(_time2scs.keys()):
        if len(_time2scs[time]) > 1:
            _first_div_time = time
            break
    # print("_first time", _first_time, "_first_div_time", _first_div_time)
    for sc in sample:
        # assert "_sample_idx" not in sc.meta
        # assert "mitosis_relative_time" not in sc.meta
        sc.meta["_sample_idx"] = idx
        sc.meta["mitosis_relative_time"] = sc.timeframe - _first_time

        sc.meta["_mitosis_first_div_time"] = _first_div_time
        if _first_div_time is None:
            sc.meta["mitosis_div_relative_time"] = None
            # sc.show(crop=True, padding=200)
            # import matplotlib.pyplot as plt
            # plt.show()
        else:
            sc.meta["mitosis_div_relative_time"] = sc.timeframe - _first_div_time

    # calculate normalized mitosis_div_relative_time across the cells in the sample
    all_mitosis_div_relative_time = np.array([sc.meta["mitosis_div_relative_time"] for sc in sample if sc.meta["mitosis_div_relative_time"] is not None])
    if len(all_mitosis_div_relative_time) == 0:
        for sc in sample:
            sc.meta["mitosis_div_relative_time_normalized"] = None
        continue
    mean_div_time = np.mean(all_mitosis_div_relative_time)
    std_div_time = np.std(all_mitosis_div_relative_time)
    if std_div_time == 0:
        std_div_time = 1

    for sc in sample:
        if sc.meta["mitosis_div_relative_time"] is None:
            continue
        else:
            sc.meta["mitosis_div_relative_time_normalized"] = (sc.meta["mitosis_div_relative_time"] - mean_div_time) / std_div_time


# report how many scs are without first_div_time
scs_without_div = [sc for sample in mitosis_sampels for sc in sample if sc.meta["mitosis_div_relative_time"] is None]
sample_ids_without_divs = set(sc.meta["_sample_idx"] for sc in scs_without_div)
print("scs_without_div", len(scs_without_div))
print("sample_ids_without_divs", sample_ids_without_divs)

print("#samples ids without divs", len(sample_ids_without_divs), "total #samples", len(mitosis_sampels))

In [ ]:
all_scs = [sc for sample in mitosis_sampels for sc in sample]

In [ ]:
processed_scs_path = out_dir / "processed_scs.json"

In [ ]:
from livecellx.core.single_cell import SingleCellStatic
from livecellx.trajectory.feature_extractors import compute_haralick_features, compute_skimage_regionprops, parallelize_compute_features
from livecellx.preprocess.utils import normalize_img_to_uint8
from livecellx.core.parallel import parallelize

def compute_skimage_regionprops_wrapper(**input):
    sc = input["sc"]
    compute_skimage_regionprops(**input)
    return sc

FORCE_RECALCULATE = False # if we force recalculate the features

if processed_scs_path.exists() and (not FORCE_RECALCULATE):
    processed_scs = SingleCellStatic.load_single_cells_json(out_dir / "processed_scs.json")
else:
    inputs = []
    for sc in all_scs:
        # features = compute_skimage_regionprops(sc, preprocess_img_func=normalize_img_to_uint8, sc_level_normalize=True)
        inputs.append({
            "sc": sc,
            "feature_key": "skimage",
            "preprocess_img_func": normalize_img_to_uint8,
            "sc_level_normalize": True,
        })

    processed_scs = parallelize(compute_skimage_regionprops_wrapper, inputs)
    from livecellx.core.single_cell import SingleCellStatic

    SingleCellStatic.write_single_cells_json(processed_scs, processed_scs_path, dataset_dir= out_dir / "datasets")

# # a for loop for computing
# import tqdm
# processed_scs = []
# for input in tqdm.tqdm(inputs):
#     sc = input["sc"]
#     compute_skimage_regionprops(**input)
#     processed_scs.append(sc)
mitosis_processed_scs = processed_scs

In [ ]:
sample_sc = mitosis_sampels[0][0]
sample_sc.meta.keys()
meta_keys = ['img_dataset_json_path', 'mask_dataset_json_path', '_sample_idx', 'mitosis_relative_time']

In [ ]:
from livecellx.core.single_cell import create_sc_table

sc_table_with_time = create_sc_table(processed_scs, add_time=True, add_sc_id=True, meta_keys=meta_keys)
cell_features_2 = create_sc_table(processed_scs, add_time=False)

In [ ]:
# save the tables
sc_table_with_time.to_csv(out_dir / "mitosis_scs_table_with_time.csv")

In [ ]:
sc_table_with_time[:2]

In [ ]:
import umap
reducer = umap.UMAP()
# drop NAN in sc_feature_table
cell_features_2 = cell_features_2.dropna(axis=1)

embedding = reducer.fit_transform(cell_features_2)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=[sc.timeframe for sc in processed_scs],
    alpha=0.5,
)
plt.title("Ignore this figure: meaningless timeframe")
plt.xlabel("UMAP_1")
plt.ylabel("UMAP_2")
# add colorbar
plt.colorbar()

In [ ]:
plt_fontsize = 14

In [ ]:
mitosis_relative_times = [sc.meta["mitosis_relative_time"] for sc in processed_scs]

# plot distribution of mitosis relative times
import seaborn as sns
plt.figure(figsize=(8, 6))
sns.displot(mitosis_relative_times, kde=True, height=6, aspect=1.5)
plt.title("Distribution of mitosis relative times", fontsize=plt_fontsize)
plt.xlabel("Mitosis relative time", fontsize=plt_fontsize)
plt.ylabel("Count", fontsize=plt_fontsize)
plt.xticks(fontsize=plt_fontsize)
plt.yticks(fontsize=plt_fontsize)
plt.show()

In [ ]:
feature_colors = [sc.meta["mitosis_relative_time"] for sc in processed_scs]
feature_colors = [min(10, x) for x in feature_colors]

In [ ]:
mitosis_sampels[37][2]

In [ ]:
import matplotlib.pyplot as plt

large_thresholds = [10, 15, 20, 25, 30, 35, 40, 45, 50]

def print_stats(processed_scs, threshold):
    large_mitosis_relative_time_scs = [sc for sc in processed_scs if sc.meta["mitosis_relative_time"] > threshold]
    tmp_unique_long_samples = set()
    for sc in large_mitosis_relative_time_scs:
        if '_labels' not in sc.meta:
            tmp_unique_long_samples.add(sc.meta['_sample_idx'])
            continue
    return len(tmp_unique_long_samples), len(large_mitosis_relative_time_scs)

# Get the number of unique long samples and total long samples for each threshold
num_unique_long_samples = []
num_total_long_scs = []
for threshold in large_thresholds:
    unique_long_samples, total_long_samples = print_stats(processed_scs, threshold)
    num_unique_long_samples.append(unique_long_samples)
    num_total_long_scs.append(total_long_samples)

# Set the figure size
plt.figure(figsize=(8, 6))

# Plot the data as a scatter plot
plt.plot(large_thresholds, num_unique_long_samples, color='blue', label='#long samples')
# plt.scatter(large_thresholds, num_total_long_samples, color='orange', label='Total Long Samples')

# Add text labels to the scatter points
for i in range(len(large_thresholds)):
    plt.text(large_thresholds[i] + 0.7, num_unique_long_samples[i] + 0.1, str(num_unique_long_samples[i]), fontsize=12, ha='center', va='bottom')
    # plt.text(large_thresholds[i], num_total_long_samples[i], str(num_total_long_samples[i]), fontsize=12, ha='center', va='bottom')

# Set the x and y axis labels with larger font size
plt.xlabel("Sample Length Threshold", fontsize=14)
plt.ylabel("Number of Samples", fontsize=14)

# Set the x and y axis tick labels with larger font size
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add a title to the plot
plt.title("Number of Samples vs. Sample Length Threshold", fontsize=plt_fontsize)
plt.xlabel("Sample Length Threshold", fontsize=plt_fontsize)
plt.ylabel("Number of Samples", fontsize=plt_fontsize)

# Add a legend to the plot
plt.legend(fontsize=12)

# remove the spines
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Show the plot
plt.show()

In [ ]:
cell_features_2.columns

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
axes[0].scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=cell_features_2["skimage_area"],
    cmap="inferno",
    alpha=0.5,
)
axes[0].set_xlabel("UMAP_1")
axes[0].set_ylabel("UMAP_2")
axes[0].set_title("Mitosis: area")
# add colorbar
# axes[0].colorbar()

axes[1].scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=feature_colors,
    cmap="inferno",
    alpha=0.5,
)
axes[1].set_xlabel("UMAP_1")
axes[1].set_ylabel("UMAP_2")
axes[1].set_title("Mitosis: relative time")

axes[2].scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=cell_features_2["skimage_eccentricity"],
    cmap="inferno",
    alpha=0.5,
)
axes[2].set_xlabel("UMAP_1")
axes[2].set_ylabel("UMAP_2")
axes[2].set_title("Mitosis: eccentricity")


axes[3].scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=cell_features_2["skimage_orientation"],
    cmap="inferno",
    alpha=0.5,
)
axes[3].set_xlabel("UMAP_1")
axes[3].set_ylabel("UMAP_2")
axes[3].set_title("Mitosis: skimage_orientation")

In [ ]:
import seaborn as sns
import pandas as pd
# Create a DataFrame with the mitosis colors and corresponding skimage areas
data = pd.DataFrame({
    "mitosis_color": feature_colors,
    "skimage_area": cell_features_2["skimage_area"]
})

# Draw a boxplot for each mitosis color
fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(x="mitosis_color", y="skimage_area", data=data, width=0.5, palette="colorblind", ax=ax)
plt.xlabel("mitosis: relative time")
plt.ylabel("area")

In [ ]:
cell_features_2.columns

In [ ]:
import seaborn as sns
import pandas as pd
# Create a DataFrame with the mitosis colors and corresponding skimage areas

def viz_boxplot(key, mitosis_times, use_abs_vals=False):
    vals = cell_features_2[key]

    if use_abs_vals:
        vals = vals.abs()
    data = pd.DataFrame({
        "mitosis: relative time": mitosis_times,
        key: vals
    })

    # Draw a boxplot for each mitosis color
    sns.boxplot(x="mitosis: relative time", y=key, data=data)
# viz_boxplot("skimage_area", clipped_mitosis_time)

"""eccentricity: float
Eccentricity of the ellipse that has the same second-moments as the region. 
The eccentricity is the ratio of the focal distance (distance between focal points) over the major axis length. 
The value is in the interval [0, 1). When it is 0, the ellipse becomes a circle."""
# viz_boxplot("skimage_eccentricity", clipped_mitosis_time, use_abs_vals=True)
# viz_boxplot("skimage_axis_minor_length", clipped_mitosis_time)
viz_boxplot("skimage_axis_major_length", feature_colors)
# viz_boxplot("skimage_orientation", clipped_mitosis_time)

In [ ]:
from livecellx.core.single_cell import create_sc_table
normalized_img_features = create_sc_table(processed_scs, normalize_features=True)
# drop na
normalized_img_features = normalized_img_features.dropna(axis=1, how="all")
reducer = umap.UMAP()
embedding = reducer.fit_transform(normalized_img_features)

fig, axes = plt.subplots(1, 3, figsize=(10, 5))
scatter_time = axes[0].scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=feature_colors,
    alpha=0.5,
)
colorbar = fig.colorbar(scatter_time, ax=axes[0], label="Time")

scatter_area = axes[1].scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=normalized_img_features["skimage_area"],
)
colorbar = fig.colorbar(scatter_area, ax=axes[1], label="Area")

scatter_src = axes[2].scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=normalized_img_features["skimage_solidity"],
)


In [ ]:
import plotly.graph_objects as go
import plotly.subplots as sp
import numpy as np

def visualize_scs_skimage_plotly(scs, embedding, feature_table, feature="", color_vals=None, title="UMAP Embedding", clip_min=None, clip_max=None, cmap="Blues", alpha=1):
    fig = sp.make_subplots(rows=1, cols=1, subplot_titles=())
    if color_vals is None:
        color_vals = feature_table[feature]
    if clip_min or clip_max:
        color_vals = np.clip(color_vals, clip_min, clip_max)
    scatter_go = go.Scatter(
        x=embedding[:, 0],
        y=embedding[:, 1],
        mode="markers",
        opacity=alpha,
        marker=dict(color=color_vals, colorscale=cmap, colorbar=dict(title=feature, x=1, yanchor="middle", len=0.5)),
        text=[f"Timeframe: {sc.timeframe}<br>sc id: {sc.id}<br>sc idx: {idx}" for idx, sc in enumerate(scs)],
    )
    scatter_go.on_click(lambda trace, points, state: display(points))
    fig.append_trace(scatter_go, row=1, col=1)
    fig.update_layout(height=600, width=800, title_text=title, clickmode="event", plot_bgcolor='rgb(1,1,1)', xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
    fig.show()

In [ ]:
processed_scs[2527].meta

In [ ]:
processed_scs[2527].show_panel()
embedding[2527]

In [ ]:
# processed_scs[2527].show(crop=True, padding=200)
processed_scs[2527].show_contour_img(padding=200)

In [ ]:
len(processed_scs) - 288

In [ ]:
is_CXA = [sc.meta["sample_src_dir"].find("CXA") != -1 for sc in processed_scs]
reversed_processed_scs = processed_scs[::-1]
reversed_is_CXA = is_CXA[::-1]
reversed_embedding = embedding[::-1]
is_CXA = np.array(is_CXA)
visualize_scs_skimage_plotly
# Build figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=embedding[is_CXA, 0],
        y=embedding[is_CXA, 1],
        marker=dict(
            color='LightSkyBlue',
            size=20,
            line=dict(
                color='Blue',
                width=2
            )
        ),
        name='CXA'
    )
)

# Add trace with large markers
fig.add_trace(
    go.Scatter(
        mode='markers',
        x=embedding[~is_CXA, 0],
        y=embedding[~is_CXA, 1],
        opacity=0.5,
        marker=dict(
            color='LightSkyBlue',
            size=5,
            line=dict(
                color='MediumPurple',
                width=5,
            )
        ),
        showlegend=True,
        name='Nikon'
    )
)
fig.update_layout(height=600, width=800, title_text="UMAP", clickmode="event", xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
fig.show()

In [ ]:
visualize_scs_skimage_plotly(processed_scs, embedding, normalized_img_features, "skimage_area", clip_max=2)

CXA mitotic cells latter phase

In [ ]:
for idx, _sc in enumerate(processed_scs):
    if _sc.meta["sample_src_dir"].find("CXA") != -1:
        if 6 <= embedding[idx][0] <= 10 and  6 < embedding[idx][1] < 20:
            _sc.show(crop=True, padding=20)
            plt.show()
            break

In [ ]:
for idx, _sc in enumerate(processed_scs):
    if _sc.meta["sample_src_dir"].find("CXA") != -1:
        if 6 <= embedding[idx][0] <= 10 and  6 < embedding[idx][1] < 20:
            print(idx, _sc.meta["sample_src_dir"], _sc.meta["mitosis_relative_time"])
            _sc.show(crop=True, padding=20)
            print("embedding:", embedding[idx])
            plt.show()

In [ ]:
# processed_scs[2681].show_panel(padding=200)
# processed_scs[2652].show_panel(padding=200)
processed_scs[2627].show_panel(padding=200)
embedding[2627]

CXA mitotic cell early subphase

In [ ]:
for idx, _sc in enumerate(processed_scs):
    if _sc.meta["sample_src_dir"].find("CXA") != -1:
        if embedding[idx][0] < 3:
            print(idx, _sc.meta["sample_src_dir"], _sc.meta["mitosis_relative_time"])

In [ ]:

# processed_scs[2684].show_panel(padding=200)
processed_scs[2682].show_panel(padding=200)
embedding[2682]

In [ ]:
import plotly.graph_objs as go
import plotly.subplots as sp
import numpy as np

scs = processed_scs
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Time", "Area"), horizontal_spacing=0.1)

mitosis_time_np = np.array(feature_colors, dtype=int)
scatter_time = go.Scatter(
    x=embedding[:, 0],
    y=embedding[:, 1],
    mode="markers",
    marker=dict(
        color=feature_colors, colorbar=dict(title="Time", x=0.45, yanchor="middle", len=0.5), 
        colorscale="blues"
        # colorscale="blackbody",
    ),
    text=[f"Timeframe: {sc.timeframe}<br>sc id: {sc.id}" for sc in scs],
    customdata=[sc.id for sc in scs],
    name="Time",
)

unique_mitosis_time_points = np.unique(mitosis_time_np)
for i, time_point in enumerate(unique_mitosis_time_points):
    if len(np.where(mitosis_time_np == time_point)) == 0:
        continue
    indices = np.where(mitosis_time_np == time_point)[0]
    mean_position = np.mean(embedding[indices], axis=0)
    x = mean_position[0]
    y = mean_position[1]
    annotation = go.layout.Annotation(
        x=x,
        y=y,
        text=str(time_point),
        showarrow=False,
        font=dict(size=13, color="black"),
        bgcolor="rgba(255,255,255,0.5)",
        bordercolor="black",
        borderwidth=0,
        borderpad=3,
        xshift=5,
        yshift=5,
    )
    fig.add_annotation(annotation)

scatter_area = go.Scatter(
    x=embedding[:, 0],
    y=embedding[:, 1],
    mode="markers",
    marker=dict(
        color=normalized_img_features["skimage_area"], colorbar=dict(title="Area", x=1, yanchor="middle", len=0.5)
    ),
    name="Area",
)
fig.append_trace(scatter_time, row=1, col=1)
fig.add_trace(scatter_area, row=1, col=2)


# add X and Y labels
# fig.update_xaxes(title_text="UMAP_1", row=1, col=1)
# fig.update_yaxes(title_text="UMAP_2", row=1, col=1)

# fig.update_xaxes(title_text="UMAP_1", row=1, col=2)
# fig.update_yaxes(title_text="UMAP_2", row=1, col=2)


fig.update_layout(height=500, width=1000, title_text="UMAP Embedding", clickmode="event", showlegend=False, xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))


unnormalized feature

In [ ]:
from livecellx.core.single_cell import create_sc_table
unnormalized_img_features = create_sc_table(processed_scs, normalize_features=False)
# drop na
unnormalized_img_features = unnormalized_img_features.dropna(axis=1, how="all")
reducer = umap.UMAP()
unnormalized_embedding = reducer.fit_transform(unnormalized_img_features)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
scatter_time = axes[0].scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=feature_colors,
)
colorbar = fig.colorbar(scatter_time, ax=axes[0], label="Time")

scatter_area = axes[1].scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=unnormalized_img_features["skimage_centroid_weighted_local-0"],
)
colorbar = fig.colorbar(scatter_area, ax=axes[1], label="Area")


In [ ]:
import plotly.graph_objs as go
import plotly.subplots as sp
import numpy as np

scs = processed_scs
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Time", "Area"))

mitosis_time_np = np.array(feature_colors, dtype=int)
scatter_time = go.Scatter(
    x=unnormalized_embedding[:, 0],
    y=unnormalized_embedding[:, 1],
    mode="markers",
    marker=dict(color=feature_colors, colorbar=dict(title="Time", x=0.45, yanchor="middle", len=0.5)),
    text=[f"Timeframe: {sc.timeframe}<br>sc id: {sc.id}<br>Mitosis: {sc.meta['mitosis_relative_time']}" for sc in scs],
    customdata=[sc.id for sc in scs],
)

unique_mitosis_time_points = np.unique(mitosis_time_np)
for i, time_point in enumerate(unique_mitosis_time_points):
    if len(np.where(mitosis_time_np == time_point)) == 0:
        continue
    indices = np.where(mitosis_time_np == time_point)[0]
    mean_position = np.mean(unnormalized_embedding[indices], axis=0)
    x = mean_position[0]
    y = mean_position[1]
    annotation = go.layout.Annotation(
        x=x,
        y=y,
        text=str(time_point),
        showarrow=False,
        font=dict(size=20, color="white"),
        xshift=5,
        yshift=5,
    )
    fig.add_annotation(annotation)

scatter_area = go.Scatter(
    x=unnormalized_embedding[:, 0],
    y=unnormalized_embedding[:, 1],
    mode="markers",
    marker=dict(color=normalized_img_features["skimage_area"], colorbar=dict(title="Area", x=1, yanchor="middle", len=0.5)),
)
fig.append_trace(scatter_time, row=1, col=1)
fig.add_trace(scatter_area, row=1, col=2)

fig.update_layout(height=500, width=1000, title_text="UMAP Embedding", clickmode="event", showlegend=False)

### PCA analysis on mitosis cells

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=30)
pca.fit(normalized_img_features)
mitosis_pca_img_features = pca.transform(normalized_img_features)

In [ ]:
mitosis_pca = pca

In [ ]:
# plot the transformed data
import matplotlib.pyplot as plt

plt.scatter(mitosis_pca_img_features[:, 0], mitosis_pca_img_features[:, 1])
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

In [ ]:
mitosis_pca_img_features.T.shape, normalized_img_features.shape


In [ ]:
def compute_dims_corr(cell_features_1: pd.DataFrame, cell_features_2: pd.DataFrame, sort_by_abs=True):
    feature_cols = cell_features_2.columns
    feature_corr_df = pd.DataFrame()
    for feature in feature_cols:
        for dim in cell_features_1.columns:
            _embedding = cell_features_1[dim]
            # suffer from NAN
            # corr = np.corrcoef(np.array(sc_feature_table[feature]), _embedding)[0, 1]

            # avoid and exclude NA values
            _tmp_df = pd.DataFrame({"embedding": _embedding, "feature": cell_features_2[feature]})
            all_corrs = _tmp_df.corr()
            corr = all_corrs["feature"].loc["embedding"]
            new_df = pd.DataFrame({"feature": [feature], "corr": [corr], "dim": [dim]})
            feature_corr_df = pd.concat([feature_corr_df, new_df], ignore_index=True)

    # sort feature_corr_df by corr
    dim2feature_corr_df = {}
    for dim in cell_features_1.columns:
        dim2feature_corr_df[dim] = feature_corr_df[feature_corr_df["dim"] == dim]\
            .sort_values(by="corr", ascending=False, key=lambda x: abs(x))
        # add a rank column
        dim2feature_corr_df[dim]["rank"] = np.arange(len(dim2feature_corr_df[dim]))
    return dim2feature_corr_df

mitosis_pca_img_features = pd.DataFrame(mitosis_pca_img_features, columns=[f"PC{i}" for i in range(mitosis_pca_img_features.shape[1])])
# compute_dims_corr(pca_img_features, normalized_img_features)

In [ ]:
# create a line chart of the explained variance
explained_var = mitosis_pca.explained_variance_ratio_
cumulative_var = np.cumsum(explained_var)
line_chart = go.Scatter(x=np.arange(1, len(explained_var) + 1), y=cumulative_var, mode="lines+markers")


# add a vertical line at the elbow point
elbow_point = 14
line_shape = go.layout.Shape(type="line", x0=elbow_point, y0=0, x1=elbow_point, y1=1, line=dict(color="red", width=2, dash="dash"))
# add a horizontal line at y=0.9
hline_shape = go.layout.Shape(type="line", x0=0, y0=0.9, x1=len(explained_var), y1=0.9, line=dict(color="red", width=2, dash="dash"))

layout = go.Layout(title="Mitosis PCA", xaxis=dict(title="Number of Components", tickvals=list(range(0, len(explained_var), 2)) + [12],), yaxis=dict(title="Cumulative Explained Variance",tickvals=[0.2, 0.4, 0.6, 0.8, 0.9, 1],), shapes=[], width=800, height=600)

fig = go.Figure(data=[line_chart], layout=layout)
fig.add_shape(line_shape)
fig.add_shape(hline_shape)
fig.show()

In [ ]:
pc_col_names = [f"PC{i}" for i in range(mitosis_pca_img_features.shape[1])]
not_mitosis_cells_pca_img_features_df = pd.DataFrame(mitosis_pca_img_features, columns=pc_col_names)
pc2df = compute_dims_corr(mitosis_pca_img_features, normalized_img_features)
print("pc2df:", pc2df["PC0"][:2])
topk_pc = 12
topk_features = 5
topk_pc_rank_df = pd.concat([pc2df[f"PC{i}"][:topk_features] for i in range(topk_pc)], ignore_index=True)
import plotly.graph_objs as go

# create a bar chart of the "corr" column
topk_pc_rank_df["abs_corr"] = topk_pc_rank_df["corr"].abs()
topk_pc_rank_df = topk_pc_rank_df.sort_values(by='abs_corr')
bar_chart = go.Bar(y=topk_pc_rank_df["feature"], x=topk_pc_rank_df["abs_corr"], orientation="h")
layout = go.Layout(
    title="Top-%d PC Rank of Mitotic Cells" % (topk_pc),
    xaxis=dict(title="Correlation"),
    yaxis=dict(title="Feature"),
    width=800,
    height=600,
)
# hide y-axis title
layout["yaxis"]["title"] = ""
fig = go.Figure(data=[bar_chart], layout=layout)
fig.show()


### Visualize non mitosis cells' label-free features  
We will next visualize our mistosis data V12.
Note that for obtaining a list of normal cells, tracking is not required. However, to reproduce V12 normal single cells that are used in our training, we need to track cells, generate samples and then get a list of normal cells for analysis usage. Tracking information may be retrained in this case for visualization purpose later.

In [ ]:
# get all scs from class_samples not in normal class
exclude_scs = []
total_non_normal_samples = 0
for class_name, samples in all_class2samples.items():
    if class_name != "normal":
        for sample in samples:
            exclude_scs.extend(sample)
            total_non_normal_samples += 1
exclude_scs = set(exclude_scs)
exclude_scs_ids = {str(sc.id) for sc in exclude_scs}

Load all V12 normal sample source.  

In [ ]:
import json
from livecellx.core.single_cell import SingleCellTrajectoryCollection
from livecellx.track.sort_tracker_utils import (
    track_SORT_bbox_from_scs
)
all_scs_json_path = ["./datasets/test_scs_EBSS_starvation/XY1/single_cells.json", "./datasets/test_scs_EBSS_starvation/XY16/single_cells.json"]
sctc = SingleCellTrajectoryCollection()
for json_path in all_scs_json_path:
    print("json path:", json_path)
    _scs = SingleCellStatic.load_single_cells_json(json_path)
    tmp_sctc = track_SORT_bbox_from_scs(_scs, raw_imgs=_scs[0].img_dataset, min_hits=3, max_age=3)
    tids = set(sctc.get_all_tids())
    if len(tids) != 0:
        max_tid = max(tids)
    else:
        max_tid = 0
    for tid, traj in tmp_sctc:
        traj.meta["src_dir"] = json_path
        traj.track_id = tid + max_tid + 1
        sctc.add_trajectory(traj)
        traj_scs = traj.get_all_scs()
        for sc in traj_scs:
            sc.meta["src_dir"] = json_path
    del tmp_sctc

all_scs = SingleCellStatic.load_single_cells_jsons(all_scs_json_path)

In [ ]:
# # set numpy seed
# seed = 0
# np.random.seed(seed)

# objective_sample_num = total_non_normal_samples * 10

# normal_frame_len_range = (3, 10)
# counter = 0
# normal_samples = []
# normal_samples_extra_info = []
# max_trial_counter = 100000
# while counter < objective_sample_num and max_trial_counter > 0:
#     # randomly select a sct from sctc
#     # generate a list of scs
#     track_id = np.random.choice(list(sctc.track_id_to_trajectory.keys()))  
#     sct = sctc.get_trajectory(track_id)
#     # randomly select a length
#     frame_len = np.random.randint(*normal_frame_len_range)
#     # generate a sample
#     times = list(sct.timeframe_to_single_cell.keys())
#     times = sorted(times)
#     if len(times) <= frame_len:
#         continue
#     start_idx = np.random.randint(0, len(times) - frame_len)
#     start_time = times[start_idx]
#     end_time = times[start_idx + frame_len - 1]

#     sub_sct = sct.subsct(start_time, end_time)

#     is_some_sc_in_exclude_scs = False
#     for time, sc in sub_sct.timeframe_to_single_cell.items():
#         # print("sc.id:", sc.id, type(sc.id))
#         if str(sc.id) in exclude_scs_ids:
#             is_some_sc_in_exclude_scs = True
#             break
#     if is_some_sc_in_exclude_scs:
#         print("some sc in the exclude scs list")
#         continue
    
#     new_sample = []
#     for time, sc in sub_sct.timeframe_to_single_cell.items():
#         new_sample.append(sc)
#     normal_samples.append(new_sample)
#     normal_samples_extra_info.append({"src_dir": sub_sct.get_all_scs()[0].meta["src_dir"]})
#     counter += 1
#     max_trial_counter -= 1

import livecellx
from livecellx.core.single_cell import sample_samples_from_sctc
# import importlib
# importlib.reload(livecellx)
# importlib.reload(livecellx.core.single_cell)

objective_sample_num = total_non_normal_samples * 10

normal_samples, normal_samples_extra_info = sample_samples_from_sctc(sctc, exclude_scs_ids=exclude_scs_ids, objective_sample_num=objective_sample_num, seed=0, length_range=(6, 10), max_trial_counter=10000)

In [ ]:
len(all_class2samples["normal"]), len(normal_samples)

In [ ]:
not_mitosis_scs = [sc for sample in normal_samples for sc in sample] + [sc for sample in all_class2samples["normal"] for sc in sample]
not_mitosis_scs = list(set(not_mitosis_scs))

print("# Not mitosis scs", len(not_mitosis_scs))

### PCA on normal_scs and calculate correlation

In [ ]:
import importlib
import livecellx
importlib.reload(livecellx.trajectory.feature_extractors)
from livecellx.trajectory.feature_extractors import compute_haralick_features, compute_skimage_regionprops, parallelize_compute_features
from livecellx.preprocess.utils import normalize_img_to_uint8
from livecellx.core.parallel import parallelize

processed_not_mitosis_scs_save_path = out_dir / "processed_normal_scs.json"
if processed_not_mitosis_scs_save_path.exists():
    not_mitosis_scs = SingleCellStatic.load_single_cells_json(out_dir / "processed_normal_scs.json")
else:
    outputs = parallelize_compute_features(not_mitosis_scs, compute_skimage_regionprops, dict(preprocess_img_func=normalize_img_to_uint8, sc_level_normalize=True))
    not_mitosis_features = outputs[0]
    not_mitosis_scs = outputs[1]
    SingleCellStatic.write_single_cells_json(not_mitosis_scs, processed_not_mitosis_scs_save_path, dataset_dir= out_dir / "datasets")

In [ ]:
import plotly.graph_objs as go
import plotly.subplots as sp
import numpy as np

not_mitosis_cells_pca = PCA(n_components=30)
not_mitosis_cells_normalized_img_features = create_sc_table(not_mitosis_scs, normalize_features=True)
not_mitosis_cells_normalized_img_features = not_mitosis_cells_normalized_img_features.dropna(axis=1, how="all")
not_mitosis_cells_pca.fit(not_mitosis_cells_normalized_img_features)
not_mitosis_cells_pca_img_features = not_mitosis_cells_pca.transform(not_mitosis_cells_normalized_img_features)


In [ ]:
cumulative_var

In [ ]:
# create a line chart of the explained variance
explained_var = not_mitosis_cells_pca.explained_variance_ratio_
cumulative_var = np.cumsum(explained_var)
line_chart = go.Scatter(x=np.arange(1, len(explained_var) + 1), y=cumulative_var, mode="lines+markers")


# add a vertical line at the elbow point
elbow_point = 12
line_shape = go.layout.Shape(type="line", x0=elbow_point, y0=0, x1=elbow_point, y1=1, line=dict(color="red", width=2, dash="dash"))
# add a horizontal line at y=0.9
hline_shape = go.layout.Shape(type="line", x0=0, y0=0.9, x1=len(explained_var), y1=0.9, line=dict(color="red", width=2, dash="dash"))

layout = go.Layout(title="Interphase PCA", xaxis=dict(title="Number of Components", tickvals=list(range(0, len(explained_var), 2)) + [12],), yaxis=dict(title="Cumulative Explained Variance",tickvals=[0.2, 0.4, 0.6, 0.8, 0.9, 1],), shapes=[], width=800, height=600)

fig = go.Figure(data=[line_chart], layout=layout)
fig.add_shape(line_shape)
fig.add_shape(hline_shape)
fig.show()

In [ ]:
visualize_scs_skimage_plotly(not_mitosis_scs, not_mitosis_cells_pca_img_features, not_mitosis_cells_normalized_img_features, feature="skimage_area", title="Normal cells: area", clip_max=1)

In [ ]:
# calculate UMAP for normal/other cells
umap_reducer = umap.UMAP()
not_mitosis_cells_umap_embedding = umap_reducer.fit_transform(not_mitosis_cells_pca_img_features)

In [ ]:
def viz_by_sc_id(scs, sc_id):
    for sc in scs:
        if sc.id == sc_id:
            sc.show(crop=True, padding=200)
viz_by_sc_id(not_mitosis_scs, 0)

In [ ]:
visualize_scs_skimage_plotly(not_mitosis_scs, not_mitosis_cells_umap_embedding, not_mitosis_cells_normalized_img_features, feature="skimage_area", title="Normal cells: area", clip_min=0, clip_max=1)

In [ ]:
visualize_scs_skimage_plotly(not_mitosis_scs, not_mitosis_cells_umap_embedding, not_mitosis_cells_normalized_img_features, feature="skimage_moments-0-1", title="Normal cells: area", clip_min=0, clip_max=1)

In [ ]:
topk_pc_rank_df["feature"] = ["mitotic-" + x for x in topk_pc_rank_df["feature"]]

In [ ]:
import plotly.graph_objs as go

pc_col_names = [f"PC{i}" for i in range(not_mitosis_cells_pca_img_features.shape[1])]
not_mitosis_cells_pca_img_features_df = pd.DataFrame(not_mitosis_cells_pca_img_features, columns=pc_col_names)
not_mitosis_pc2df = compute_dims_corr(not_mitosis_cells_pca_img_features_df, not_mitosis_cells_normalized_img_features)

topk_pc = 12
topk_features = 5
not_mitosis_topk_pc_rank_df = pd.concat([not_mitosis_pc2df[f"PC{i}"][:topk_features] for i in range(topk_pc)], ignore_index=True)

# create a bar chart of the "corr" column
not_mitosis_topk_pc_rank_df["abs_corr"] = not_mitosis_topk_pc_rank_df["corr"].abs()
not_mitosis_topk_pc_rank_df = not_mitosis_topk_pc_rank_df.sort_values(by='abs_corr')

# topk_pc_rank_df["color"] = "red"
# not_mitosis_topk_pc_rank_df["color"] = "blue"
topk_pc_rank_df["color"] = "green"
not_mitosis_topk_pc_rank_df["color"] = "purple"

combined_pc_rank_df = pd.concat([topk_pc_rank_df, not_mitosis_topk_pc_rank_df], ignore_index=True)

# Replace "skimage_" with ""
combined_pc_rank_df["feature"] = combined_pc_rank_df["feature"].str.replace("skimage_", "")
# bar_chart = go.Bar(y=not_mitosis_topk_pc_rank_df["feature"], x=not_mitosis_topk_pc_rank_df["abs_corr"], orientation="h")
# bar_chart = go.Bar(y=topk_pc_rank_df["feature"], x=not_mitosis_topk_pc_rank_df["abs_corr"], orientation="h", marker=dict(color="red"))

combined_pc_rank_df = combined_pc_rank_df.sort_values(by="abs_corr")
combined_pc_rank_df.to_csv("./tmp/combined_pc_rank_df.csv")
# combined_pc_rank_df = combined_pc_rank_df[-40:]
bar_chart = go.Bar(y=combined_pc_rank_df["feature"], x=combined_pc_rank_df["abs_corr"], orientation="h", marker=dict(color=combined_pc_rank_df["color"]))

layout = go.Layout(
    title="Top-%d PC Rank of Interphase and Mitotic Cells" % (topk_pc),
    xaxis=dict(title="Correlation"),
    yaxis=dict(title="Feature"),
    width=800,
    height=1600,
)
layout["yaxis"]["title"] = ""
fig = go.Figure(data=[bar_chart], layout=layout)
fig.show()


In [ ]:
import plotly
import plotly.io as pio
# import importlib
# importlib.reload(plotly.io)

plotly.io.write_image(fig, './tmp/Figure-PCA-top-pc-features.png', scale=2)

In [ ]:
# find all features in topk_pc_rank_df but not in not_mitosis_topk_pc_rank_df

mitosis_features = topk_pc_rank_df["feature"].unique()
not_mitosis_features = not_mitosis_topk_pc_rank_df["feature"].unique()
not_mitosis_features = set(not_mitosis_features)
mitosis_features = [feature.replace("mitotic-", "") for feature in mitosis_features]
mitosis_features = set(mitosis_features)

mitosis_features - not_mitosis_features

In [ ]:
[feature for feature in mitosis_features if feature.find("area") != -1], [feature for feature in not_mitosis_features if feature.find("area") != -1]

In [ ]:
mitosis_normalized_feature = normalized_img_features
not_mitosis_cells_normalized_img_features, normalized_img_features
assert set(mitosis_normalized_feature.columns) == set(not_mitosis_cells_normalized_img_features)

### Define the labels of mitosis and interphase cells
0 - mitosis  
1 - interphase  

In [ ]:
# merge the two tables and add a label according to the class
mitosis_normalized_feature["label"] = 0
not_mitosis_cells_normalized_img_features["label"] = 1

all_cells_merged_features_df = pd.concat([mitosis_normalized_feature, not_mitosis_cells_normalized_img_features], ignore_index=True)

In [ ]:
len(all_cells_merged_features_df.columns)

In [ ]:
# train a model to predict mitosis relative time

assert len(normalized_img_features) == len(mitosis_processed_scs) and len(not_mitosis_cells_normalized_img_features) == len(not_mitosis_scs)


### Predict mitosis relative time

In [ ]:
for sc in mitosis_processed_scs:
    assert "mitosis_div_relative_time" in sc.meta, sc

In [ ]:
normalized_img_features["mitosis_relative_time"] = [sc.meta["mitosis_relative_time"] for sc in mitosis_processed_scs]
normalized_img_features["mitosis_div_relative_time"] = [sc.meta["mitosis_div_relative_time"] for sc in mitosis_processed_scs]
normalized_img_features["mitosis_div_relative_time_normalized"] = [sc.meta["mitosis_div_relative_time_normalized"] for sc in mitosis_processed_scs]

# clip the mitosis relative time to 10
normalized_img_features["mitosis_relative_time"] = np.clip(normalized_img_features["mitosis_relative_time"], 0, 10)
normalized_img_features["mitosis_div_relative_time"] = np.clip(normalized_img_features["mitosis_div_relative_time"], -10, 10)
normalized_img_features["mitosis_div_relative_time_normalized"] = np.clip(normalized_img_features["mitosis_div_relative_time_normalized"], -10, 10)
not_mitosis_cells_normalized_img_features["mitosis_relative_time"] = [0 for sc in not_mitosis_scs]

all_cells_merged_features_df = pd.concat([normalized_img_features, not_mitosis_cells_normalized_img_features], ignore_index=True)

In [ ]:
mitosis_normalized_feature["relative_time"] = [sc.meta["mitosis_relative_time"] for sc in mitosis_processed_scs]


min_relative_time, max_relative_time = 1, 15
filtered_mitosis_normalized_feature = mitosis_normalized_feature[mitosis_normalized_feature["relative_time"] >= min_relative_time]
filtered_mitosis_normalized_feature = filtered_mitosis_normalized_feature[mitosis_normalized_feature["relative_time"] <= max_relative_time]
filtered_mitosis_normalized_feature.drop("relative_time", axis=1, inplace=True)
# filtered_mitosis_normalized_feature.drop("mitosis_relative_time", axis=1, inplace=True)

In [ ]:
len(normalized_img_features), len(filtered_mitosis_normalized_feature)

In [ ]:
filtered_mitosis_normalized_feature.columns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

# define features (X) and target (y)
# X = all_cells_merged_features_df.drop('mitosis_relative_time', axis=1)
# y = all_cells_merged_features_df['mitosis_relative_time']

# X = normalized_img_features.drop('mitosis_relative_time', axis=1)
# y = normalized_img_features['mitosis_relative_time']
X = filtered_mitosis_normalized_feature.drop('mitosis_relative_time', axis=1).drop("mitosis_div_relative_time", axis=1).drop("mitosis_div_relative_time_normalized", axis=1)
y = filtered_mitosis_normalized_feature['mitosis_relative_time']

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train a linear regression model
# lr_model = LinearRegression()
mitosis_time_lr_model = Lasso(alpha=0.01, max_iter=1000000)
mitosis_time_lr_model.fit(X_train, y_train)


# print training loss
y_pred_train = mitosis_time_lr_model.predict(X_train)
train_loss = np.sqrt(mean_squared_error(y_train, y_pred_train))
print("MSE train loss:", train_loss)

# make predictions on the testing data
y_pred = mitosis_time_lr_model.predict(X_test)

# compute the RMSE of the predictions
rmse_mitosis_time = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: ", rmse_mitosis_time)

Predict on div_time instead of mitosis relative time

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
# define features (X) and target (y)
# X = all_cells_merged_features_df.drop('mitosis_relative_time', axis=1)
# y = all_cells_merged_features_df['mitosis_relative_time']

# X = normalized_img_features.drop('mitosis_relative_time', axis=1)
# y = normalized_img_features['mitosis_relative_time']

no_na_filtered_mitosis_normalized_feature = filtered_mitosis_normalized_feature.dropna()


# # filter by relative division time
# no_na_filtered_mitosis_normalized_feature = no_na_filtered_mitosis_normalized_feature[no_na_filtered_mitosis_normalized_feature["mitosis_div_relative_time_normalized"] < 0]

X = no_na_filtered_mitosis_normalized_feature.drop('mitosis_relative_time', axis=1)\
    .drop("mitosis_div_relative_time", axis=1)\
    .drop("mitosis_div_relative_time_normalized", axis=1)
# y = no_na_filtered_mitosis_normalized_feature["mitosis_div_relative_time"].abs()
y = no_na_filtered_mitosis_normalized_feature["mitosis_div_relative_time_normalized"].abs()

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train a linear regression model
# lr_model = LinearRegression()
div_time_lr_model = Lasso(alpha=0.0005, max_iter=1000000)
# div_time_lr_model = SVR(kernel='rbf', C=1.0, epsilon=0.2)

div_time_lr_model.fit(X_train, y_train)

# make predictions on the testing data
y_pred = div_time_lr_model.predict(X_test)

# print training loss
y_pred_train = div_time_lr_model.predict(X_train)
train_loss = np.sqrt(mean_squared_error(y_train, y_pred_train))
print("MSE train loss:", train_loss)

# compute the RMSE of the predictions
rmse_div_time = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: ", rmse_div_time)

In [ ]:
import matplotlib.pyplot as plt

lr_model = div_time_lr_model
# make predictions on the training data
y_train_pred = lr_model.predict(X_train)

# compute the RMSE of the predictions
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# create a scatter plot of actual vs. predicted values
plt.figure(figsize=(10, 12), dpi=300)
plt.scatter(y_train, y_train_pred, color='blue', label=f'Train, RMSE={train_rmse:.2f}', alpha=0.1)
plt.scatter(y_test, y_pred, color='red', label=f'Test, RMSE={test_rmse:.2f}', alpha=0.1)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=3, label='Perfect prediction')
# plt.plot([0, 5], [0, 5], 'k--', lw=3, label='Perfect prediction')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted Values')
plt.xlim(0, 2.5)  # set the x-axis limits
plt.ylim(0, 2.5)  # set the y-axis limits
plt.legend()
plt.show()

Predict on HeLa cells

In [ ]:
# import matplotlib.pyplot as plt

# # make predictions on the testing data
# y_pred = lr_model.predict(X_test)

# # create a scatter plot of actual vs predicted values

# plt.scatter(y_test, y_pred)
# plt.xlabel('Actual Values')
# plt.ylabel('Predicted Values')
# plt.title('Actual vs Predicted Values')
# plt.show()

In [ ]:
# from sklearn.svm import SVR

# # define features (X) and target (y)
# X = normalized_img_features.drop('mitosis_relative_time', axis=1)
# y = normalized_img_features['mitosis_relative_time']

# # split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # train a SVR model
# svr_model = SVR(kernel='linear')  # you can change the kernel as needed
# svr_model.fit(X_train, y_train)

# # make predictions on the testing data
# y_pred = svr_model.predict(X_test)

# # compute the RMSE of the predictions
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print("RMSE: ", rmse)

In [ ]:
len(mitosis_normalized_feature), len(mitosis_processed_scs), len(not_mitosis_scs), len(not_mitosis_cells_normalized_img_features)

## Logistic regression
Filter according to mitosis relative time

In [ ]:
mitosis_normalized_feature["relative_time"] = [sc.meta["mitosis_relative_time"] for sc in mitosis_processed_scs]

min_relative_time, max_relative_time = 1, 15
filtered_mitosis_normalized_feature = mitosis_normalized_feature[mitosis_normalized_feature["relative_time"] >= min_relative_time]

filtered_mitosis_normalized_feature = filtered_mitosis_normalized_feature[mitosis_normalized_feature["relative_time"] <= max_relative_time]
filtered_mitosis_normalized_feature.drop("relative_time", axis=1, inplace=True)

In [ ]:
len(mitosis_normalized_feature)

In [ ]:
filtered_mitosis_normalized_feature.drop("mitosis_relative_time", axis=1, inplace=True)

In [ ]:
if "mitosis_relative_time" in not_mitosis_cells_normalized_img_features:
    not_mitosis_cells_normalized_img_features.drop("mitosis_relative_time", axis=1, inplace=True)

In [ ]:
all_cells_merged_features_df.dropna()["label"].unique()

In [ ]:
", ".join(all_cells_merged_features_df.columns)

In [ ]:
# merge the two tables and add a label according to the class
filtered_mitosis_normalized_feature["label"] = 0
not_mitosis_cells_normalized_img_features["label"] = 1

all_cells_merged_features_df = pd.concat([filtered_mitosis_normalized_feature, not_mitosis_cells_normalized_img_features], ignore_index=True)

all_cells_merged_features_df = all_cells_merged_features_df.drop("mitosis_relative_times", axis=1, errors="ignore").drop("relative_time", errors="ignore", axis=1).drop("mitosis_div_relative_time_normalized", axis=1).drop("mitosis_div_relative_time", axis=1)

assert "mitosis_relative_time" not in all_cells_merged_features_df.columns
assert "relative_time" not in all_cells_merged_features_df.columns
# Train a linear regression model to predict the label

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# split the data into train and test
X = all_cells_merged_features_df.dropna().drop(columns=["label"])
y = all_cells_merged_features_df.dropna()["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.to_csv(out_dir / "X_train.csv", index=False)
X_test.to_csv(out_dir / "X_test.csv", index=False)
y_train.to_csv(out_dir / "y_train.csv", index=False)
y_test.to_csv(out_dir / "y_test.csv", index=False)

all_df = pd.concat([mitosis_normalized_feature, not_mitosis_cells_normalized_img_features], ignore_index=True)
all_df.to_csv(out_dir / "all_df_with_times.csv", index=False)

In [ ]:
all_df.columns

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def lr_model():
    # train a logistic regression model
    # lr_model = LogisticRegression(n_jobs=-1, solver='saga', max_iter=100000)
    lr_model = LogisticRegression(n_jobs=-1, solver='newton-cg', max_iter=10000, penalty=None)
    lr_model.fit(X_train, y_train)


    y_pred = lr_model.predict(X_train)

    # compute the accuracy
    accuracy = accuracy_score(y_train, y_pred)
    print ("#" * 40, "Train", "#" * 40)
    print("Accuracy:", accuracy)

    # print classification report
    print("Classification Report:")
    print(classification_report(y_train, y_pred, digits=4))

    # print confusion matrix
    print("Confusion Matrix:")
    print(confusion_matrix(y_train, y_pred))

    print("#" * 40, "Test", "#" * 40)
    y_pred = lr_model.predict(X_test)

    # compute the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    # print classification report
    print("Classification Report:")
    print(classification_report(y_test, y_pred, digits=4))

    # print confusion matrix
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# train a logistic regression model
# lr_model = LogisticRegression(n_jobs=-1, solver='saga', max_iter=100000)
lr_model = LogisticRegression(n_jobs=-1, solver='newton-cg', max_iter=10000, penalty=None)
lr_model.fit(X_train, y_train)


y_pred = lr_model.predict(X_train)

# compute the accuracy
accuracy = accuracy_score(y_train, y_pred)
print ("#" * 40, "Train", "#" * 40)
print("Accuracy:", accuracy)

# print classification report
print("Classification Report:")
print(classification_report(y_train, y_pred, digits=4))

# print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_train, y_pred))

print("#" * 40, "Test", "#" * 40)
y_pred = lr_model.predict(X_test)

# compute the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, digits=4))

# print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


## Load and predict on hela cells

In [ ]:
hela_json_dirs = [
    Path(r"../datasets/mitosis-annotations-2023/DIC-C2DH-HeLa/01_annotation"),
    Path(r"../datasets/mitosis-annotations-2023/DIC-C2DH-HeLa/02_annotation")
]

hela_all_class2samples, hela_all_class2sample_extra_info = load_all_json_dirs(hela_json_dirs)
hela_scs = []
for cls in hela_all_class2samples:
    for sample in hela_all_class2samples[cls]:
        hela_scs.extend(sample)
print("#hela scs:", len(hela_scs))

In [ ]:
all_hela_cells_path = "../datasets/mitosis-annotations-2023/DIC-C2DH-HeLa/01_annotation/single_cells.json"
all_hela_scs = SingleCellStatic.load_single_cells_json(all_hela_cells_path)

mitosis_hela_cells = [sc for sc in sample for sample in hela_all_class2samples["mitosis"]]
mitosis_hela_cell_ids = [sc.id for sc in mitosis_hela_cells]
print(f"{len(all_hela_scs)} loaded")
all_non_mitosis_hela_scs = [sc for sc in all_hela_scs if sc.id not in mitosis_hela_cell_ids]

print(f"{len(all_non_mitosis_hela_scs)} interphase cells")

In [ ]:
from livecellx.trajectory.feature_extractors import parallelize_compute_features
# compute features for hela_scs
non_mitosis_hela_processed_scs_save_path = out_dir / "processed_interphase_hela_scs.json"
if non_mitosis_hela_processed_scs_save_path.exists():
    processed_non_mitosis_hela_scs = SingleCellStatic.load_single_cells_json(non_mitosis_hela_processed_scs_save_path)
else:
    _, processed_non_mitosis_hela_scs = parallelize_compute_features(all_non_mitosis_hela_scs, compute_skimage_regionprops, {"feature_key": "skimage",
            "preprocess_img_func": normalize_img_to_uint8,
            "sc_level_normalize": True,
        })
    SingleCellStatic.write_single_cells_json(processed_non_mitosis_hela_scs, non_mitosis_hela_processed_scs_save_path, dataset_dir= out_dir / "datasets")

non_mitosis_processed_hela_scs = processed_non_mitosis_hela_scs

In [ ]:
from livecellx.trajectory.feature_extractors import parallelize_compute_features
# compute features for hela_scs
hela_processed_scs_save_path = out_dir / "processed_hela_scs.json"
if hela_processed_scs_save_path.exists():
    processed_hela_scs = SingleCellStatic.load_single_cells_json(hela_processed_scs_save_path)
else:
    _, processed_hela_scs = parallelize_compute_features(hela_scs, compute_skimage_regionprops, {"feature_key": "skimage",
            "preprocess_img_func": normalize_img_to_uint8,
            "sc_level_normalize": True,
        })
    SingleCellStatic.write_single_cells_json(processed_hela_scs, hela_processed_scs_save_path, dataset_dir= out_dir / "datasets")

mitosis_hela_cells = processed_hela_scs

In [ ]:
hela_mitosis_normalized_img_features = create_sc_table(processed_hela_scs, normalize_features=True)

In [ ]:
set(filtered_mitosis_normalized_feature.columns).difference(set(hela_mitosis_normalized_img_features.columns))

In [ ]:
set(hela_mitosis_normalized_img_features.columns).difference(set(filtered_mitosis_normalized_feature.columns))

In [ ]:
len(set(hela_mitosis_normalized_img_features.columns)), len(set(filtered_mitosis_normalized_feature.columns))

In [ ]:
lr_features = lr_model.feature_names_in_
hela_mitosis_normalized_img_features = hela_mitosis_normalized_img_features[lr_features]

In [ ]:
# assert len(set(hela_mitosis_normalized_img_features.columns)) == (len(set(filtered_mitosis_normalized_feature.columns)) - 1) # minus label

In [ ]:
non_mitosis_hela_mitosis_normalized_img_features = create_sc_table(non_mitosis_processed_hela_scs, normalize_features=True)
hela_non_mitosis_normalized_img_features = non_mitosis_hela_mitosis_normalized_img_features[lr_features]

In [ ]:
hela_all = pd.concat([hela_mitosis_normalized_img_features, hela_non_mitosis_normalized_img_features], ignore_index=True)
hela_all_y_labels = [0 for _ in range(len(hela_mitosis_normalized_img_features))] + [1 for _ in range(len(hela_non_mitosis_normalized_img_features))]

hela_preds = lr_model.predict(hela_all)
accuracy = lr_model.score(hela_all, hela_all_y_labels)
print("Accuracy:", accuracy * 100)

# print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(hela_all_y_labels, hela_preds))


In [ ]:
all_hela_cells_path = "../datasets/mitosis-annotations-2023/DIC-C2DH-HeLa/01_annotation/single_cells.json"
all_hela_scs = SingleCellStatic.load_single_cells_json(all_hela_cells_path)

mitosis_hela_cell_ids = [sc.id for sc in mitosis_hela_cells]
print(f"{len(all_hela_scs)} loaded")
all_non_mitosis_hela_scs = [sc for sc in all_hela_scs if sc.id not in mitosis_hela_cell_ids]

print(f"{len(all_non_mitosis_hela_scs)} interphase cells")

In [ ]:
from livecellx.trajectory.feature_extractors import parallelize_compute_features
# compute features for hela_scs
non_mitosis_hela_processed_scs_save_path = out_dir / "processed_interphase_hela_scs.json"
if non_mitosis_hela_processed_scs_save_path.exists():
    processed_non_mitosis_hela_scs = SingleCellStatic.load_single_cells_json(non_mitosis_hela_processed_scs_save_path)
else:
    _, processed_non_mitosis_hela_scs = parallelize_compute_features(all_non_mitosis_hela_scs, compute_skimage_regionprops, {"feature_key": "skimage",
            "preprocess_img_func": normalize_img_to_uint8,
            "sc_level_normalize": True,
        })
    SingleCellStatic.write_single_cells_json(processed_non_mitosis_hela_scs, non_mitosis_hela_processed_scs_save_path, dataset_dir= out_dir / "datasets")

non_mitosis_processed_hela_scs = processed_non_mitosis_hela_scs

In [ ]:
non_mitosis_hela_mitosis_normalized_img_features = create_sc_table(non_mitosis_processed_hela_scs, normalize_features=True)
hela_non_mitosis_normalized_img_features = non_mitosis_hela_mitosis_normalized_img_features[lr_features]

In [ ]:
len(hela_non_mitosis_normalized_img_features)

In [ ]:
hela_all = pd.concat([hela_mitosis_normalized_img_features, hela_non_mitosis_normalized_img_features], ignore_index=True)
hela_all_y_labels = [0 for _ in range(len(hela_mitosis_normalized_img_features))] + [1 for _ in range(len(hela_non_mitosis_normalized_img_features))]
hela_all.to_csv(out_dir / "hela_all.csv", index=False)
hela_all_y_labels = pd.Series(hela_all_y_labels)
hela_all_y_labels.to_csv(out_dir / "hela_all_y_labels.csv", index=False)

In [ ]:

accuracy = lr_model.score(hela_all, hela_all_y_labels)
print("Accuracy:", accuracy * 100)

In [ ]:
hela_all_predicted_labels = lr_model.predict(hela_all)
print("Classification Report:")
print(classification_report(hela_all_y_labels, hela_all_predicted_labels, digits=4))